In [4]:
import pandas as pd
import numpy as np
import os
import geopy.distance

## Customer & Gender By Month: 2013-2018

In [72]:
months_2015_2017 = []

for year in ['2015', '2016','2017']:
    for x in range(12):
        if x+1<10:
            month = year + str(0)+str(x+1)
        if x+1>=10:
            month = year + str(x+1)
        months_2015_2017.append(month)

months_2013 = ['2013-08','2013-09','2013-10','2013-11','2013-12']
months_2014 = ['2014-01','2014-02','2014-03','2014-04','2014-05','2014-06','2014-07','2014-08', '201409','201410','201411','201412']
months_2018 = ['201801','201802']
months_list = months_2013 + months_2014 + months_2015_2017 + months_2018

In [ ]:
#Create starter start/end station DataFrames which I'll use later on in the Station section

csv_path = os.path.join('data', '2013-07 - Citi Bike trip data.csv')
starter_csv = pd.read_csv(csv_path)

start_station = starter_csv.groupby('start station id').size().reset_index(name='2013-07').rename(columns={'start station id':'station_id'})
end_station = starter_csv.groupby('end station id').size().reset_index(name='2013-07').rename(columns={'end station id':'station_id'})

end_station.head()

In [73]:
subscribers = []

#Read in every monthly CSV file
for x in months_list:
    
    #if statements to account for the inconsistent naming of the original csv files
    if len(x) == 7:
        filename = x+' - Citi Bike trip data.csv'
    elif '2018' in x:
        filename = x+'_citibikenyc_tripdata.csv'
    else:   
        filename = x+'-citibike-tripdata.csv'
    
    csv_path = os.path.join('data', filename)
    csv = pd.read_csv(csv_path)
    
    #Convert columns names to lowercase and remove spaces to ensure uniform headers
    csv.columns = csv.columns.str.lower()
    csv.columns = csv.columns.str.replace(' ','')
    
    #Count the conditional values in columns
    trips = len(csv)
    customer = len(csv[csv["usertype"]=='Customer'])
    subscriber = len(csv[csv["usertype"]=='Subscriber'])
    unkn_gender = len(csv[csv["gender"]==0])
    male = len(csv[csv["gender"]==1])
    female = len(csv[csv["gender"]==2])

    #Create dictionary of all the values
    month_dict = {
        'month':x, 
        'trips':trips,
        'customer': customer, 
        'subscriber': subscriber,
        'unkn_gender': unkn_gender,
        'male':male,
        'female':female
    }
    
    #print to track the processing of each csv file
    print(f"{x}")
    
    #Append that month's dictionary to the subscribers list of dictionaries
    subscribers.append(month_dict)
    
    Build the Start Station DataFrame
    x_start_station = csv.groupby('start station id').size().reset_index(name=x).rename(columns={'start station id':'station_id'})
    start_station = pd.merge(start_station, x_start_station, on = 'station_id')
    
    #Build the End Station DataFrame
    x_end_station = csv.groupby('end station id').size().reset_index(name=x).rename(columns={'end station id':'station_id'})
    end_station = pd.merge(end_station, x_end_station, on = 'station_id')

2013-08
2013-09
2013-10
2013-11
2013-12
2014-01
2014-02
2014-03
2014-04
2014-05
2014-06
2014-07
2014-08
201409
201410
201411
201412
201501
201502
201503
201504
201505
201506
201507
201508
201509
201510
201511
201512
201601
201602
201603
201604
201605
201606
201607
201608
201609
201610
201611
201612
201701
201702
201703
201704
201705
201706
201707
201708
201709
201710
201711
201712
201801
201802


In [74]:
#Convert the subscribers list of dictionaries into a Dataframe
month_df = pd.DataFrame(subscribers)
month_df = month_df[['month','trips','customer','subscriber','unkn_gender','male','female']]
month_df

,month,trips,customer,subscriber,unkn_gender,male,female
0,2013-08,1001958,186056,815902,186096,616914,198948
1,2013-09,1034359,145169,889190,145227,669732,219400
2,2013-10,1037712,97446,940266,97498,714293,225921
3,2013-11,675774,43973,631801,44017,487386,144371
4,2013-12,443966,19381,424585,19394,335587,88985
5,2014-01,300400,7254,293146,7269,236106,57025
6,2014-02,224736,6717,218019,6731,176526,41479
7,2014-03,439117,22882,416235,22925,330568,85624
8,2014-04,670780,63047,607733,63112,473395,134273
9,2014-05,866117,107551,758566,107611,583696,174810


In [77]:
#Write to CSV
month_df.to_csv('month.csv')